In [22]:
import numpy as np

def read_flowshop_data(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
    
    # Đọc số lượng công việc và số lượng máy
    nb_jobs, nb_machines = map(int, lines[0].strip().split())
    
    # Khởi tạo mảng dữ liệu
    data = np.zeros((nb_machines, nb_jobs), dtype=int)
    
    # Đọc dữ liệu từ file
    for job_id, line in enumerate(lines[1:]):
        parts = line.strip().split()
        for i in range(nb_machines):
            data[i][job_id] = int(parts[2 * i + 1])
    
    return data, nb_jobs, nb_machines

# Ví dụ sử dụng hàm đọc file
file_path = 'data3.txt'
data, nb_jobs, nb_machines = read_flowshop_data(file_path)

# Định nghĩa các hàm của bạn
class Flowshop:
    def __init__(self, data):
        self.data = data
        self.nb_machines = data.shape[0]
        self.nb_jobs = data.shape[1]

    def _get_makespan(self, seq, data):
        c = np.zeros((self.nb_machines, len(seq)), dtype=object)
        c[0][0] = (0, data[0][seq[0]])
        for m_id in range(1, self.nb_machines):
            s_t = c[m_id-1][0][1]
            e_t = s_t + data[m_id][0]
            c[m_id][0] = (s_t, e_t)
        if len(seq) > 1:
            for i, job_id in enumerate(seq[1:]):
                s_t = c[0][i][1]
                e_t = s_t + data[0][job_id]
                c[0][i+1] = (s_t, e_t)
                for m_id in range(1, self.nb_machines):
                    s_t = max(c[m_id][i][1], c[m_id-1][i+1][1])
                    e_t = s_t + data[m_id][job_id]
                    c[m_id][i+1] = (s_t, e_t)
        return c[self.nb_machines-1][-1][1]

    def neh_heuristic(self):
        sums = []
        default_timer = None
        if sys.platform == "win32":
            default_timer = time.perf_counter
        else:
            default_timer = time.time
        s = default_timer.__call__()

        for job_id in range(self.nb_jobs):
            p_ij = sum([self.data[j][job_id] for j in range(self.nb_machines)])
            sums.append((job_id, p_ij))
        sums.sort(key=lambda x: x[1], reverse=True)
        seq = []
        for job in sums:
            cands = []
            for i in range(0, len(seq) + 1):
                cand = seq[:i] + [job[0]] + seq[i:]
                cands.append((cand, self._get_makespan(cand, self.data)))
            seq = min(cands, key=lambda x: x[1])[0]

        e = default_timer.__call__()

        schedules = np.zeros((self.nb_machines, self.nb_jobs), dtype=dict)
        # schedule first job alone first
        task = {"name": "job_{}".format(seq[0]+1), "start_time": 0, "end_time": self.data[0][seq[0]]}
        schedules[0][0] = task
        for m_id in range(1, self.nb_machines):
            start_t = schedules[m_id-1][0]["end_time"]
            end_t = start_t + self.data[m_id][0]
            task = {"name": "job_{}".format(seq[0]+1), "start_time": start_t, "end_time": end_t}
            schedules[m_id][0] = task

        for index, job_id in enumerate(seq[1:]):
            start_t = schedules[0][index]["end_time"]
            end_t = start_t + self.data[0][job_id]
            task = {"name": "job_{}".format(job_id+1), "start_time": start_t, "end_time": end_t}
            schedules[0][index+1] = task
            for m_id in range(1, self.nb_machines):
                start_t = max(schedules[m_id][index]["end_time"], schedules[m_id-1][index+1]["end_time"])
                end_t = start_t + self.data[m_id][job_id]
                task = {"name": "job_{}".format(job_id+1), "start_time": start_t, "end_time": end_t}
                schedules[m_id][index+1] = task
        max_mkspn = int(schedules[self.nb_machines-1][-1]["end_time"])
        
        t_t = e - s
        return seq, schedules, max_mkspn, t_t

# Tạo đối tượng Flowshop và gọi hàm neh_heuristic
flowshop = Flowshop(data)
seq, schedules, max_mkspn, t_t = flowshop.neh_heuristic()

print("Sequence:", seq)
print("Schedules:", schedules)
print("Makespan:", max_mkspn)
print("Time taken:", t_t)


Sequence: [8, 13, 19, 15, 3, 16, 18, 14, 5, 4, 17, 10, 9, 12, 6, 0, 11, 7, 2, 1]
Schedules: [[{'name': 'job_9', 'start_time': 0, 'end_time': 2}
  {'name': 'job_14', 'start_time': 2, 'end_time': 21}
  {'name': 'job_20', 'start_time': 21, 'end_time': 34}
  {'name': 'job_16', 'start_time': 34, 'end_time': 43}
  {'name': 'job_4', 'start_time': 43, 'end_time': 47}
  {'name': 'job_17', 'start_time': 47, 'end_time': 52}
  {'name': 'job_19', 'start_time': 52, 'end_time': 86}
  {'name': 'job_15', 'start_time': 86, 'end_time': 167}
  {'name': 'job_6', 'start_time': 167, 'end_time': 220}
  {'name': 'job_5', 'start_time': 220, 'end_time': 269}
  {'name': 'job_18', 'start_time': 269, 'end_time': 277}
  {'name': 'job_11', 'start_time': 277, 'end_time': 305}
  {'name': 'job_10', 'start_time': 305, 'end_time': 402}
  {'name': 'job_13', 'start_time': 402, 'end_time': 427}
  {'name': 'job_7', 'start_time': 427, 'end_time': 515}
  {'name': 'job_1', 'start_time': 515, 'end_time': 592}
  {'name': 'job_12',